In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-academia.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-wiki.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-cleaned-news.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-iium.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-parliament.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-watpadd.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-cleaned-common-crawl.txt

files = ['filtered-dumping-academia.txt',
        'filtered-dumping-wiki.txt',
        'dumping-cleaned-news.txt',
        'dumping-iium.txt',
        'dumping-parliament.txt',
        'dumping-watpadd.txt',
        'filtered-dumping-cleaned-common-crawl.txt']

In [2]:
import os
import string

vocabs = list(string.ascii_lowercase + string.digits) + [' ']
directory = '/home/husein/pure-text'

In [3]:
import unicodedata
import re
import itertools

def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [4]:
texts = []

for f in files:
    print(f)
    with open(os.path.join(directory, f)) as fopen:
        text = list(filter(None, fopen.read().split('\n')))
        texts.extend(text)

filtered-dumping-academia.txt
filtered-dumping-wiki.txt
dumping-cleaned-news.txt
dumping-iium.txt
dumping-parliament.txt
dumping-watpadd.txt
filtered-dumping-cleaned-common-crawl.txt


In [5]:
import json

with open('bahasa-asr-train-combined.json') as fopen:
    data = json.load(fopen)
    
texts.extend(data['Y'])

with open('bahasa-asr-test.json') as fopen:
    data = json.load(fopen)
    
texts.extend(data['Y'])

In [6]:
import mp
from tqdm import tqdm

def loop(texts):
    texts, _ = texts
    cleaned_texts = []
    for i in tqdm(range(len(texts))):
        t = preprocessing_text(texts[i])
        if len(t):
            cleaned_texts.append(t)
    return cleaned_texts

cleaned_texts = mp.multiprocessing(texts, loop, cores = 16)

100%|██████████| 3490847/3490847 [06:54<00:00, 8422.59it/s] 


In [7]:
len(texts), len(cleaned_texts)

(55853565, 55826054)

In [8]:
with open('singlish.txt') as fopen:
    s = list(filter(None, fopen.read().split('\n')))
len(s)

19870767

In [10]:
cleaned_texts.extend(s)

In [11]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(cleaned_texts))

In [12]:
!./kenlm/build/bin/lmplz --text text.txt --arpa out.arpa -o 3 --prune 0 1 1

=== 1/5 Counting and sorting n-grams ===
Reading /home/husein/malaya-speech/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 805900650 types 4918146
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:59017752 2:75162148864 3:140929040384
Statistics:
1 4918146 D1=0.748703 D2=1.03679 D3+=1.28125
2 25120009/64555612 D1=0.76653 D2=1.06148 D3+=1.31234
3 61233336/212507344 D1=0.717087 D2=1.30735 D3+=1.42433
Memory estimate for binary LM:
type      MB
probing 1748 assuming -p 1.5
probing 1910 assuming -r models -p 1.5
trie     842 without quantization
trie     533 assuming -q 8 -b 8 quantization 
trie     788 assuming -a 22 array pointer compression
trie     480 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:59

In [13]:
!./kenlm/build/bin/build_binary -q 8 -b 7 -a 256 trie out.arpa out-mixed.trie.klm

Reading out.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Identifying n-grams omitted by SRI
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Quantizing
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Writing trie
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [14]:
!rm text.txt out.arpa singlish.txt sg-news.txt manglish.json singlish-text.json

In [15]:
b2_application_key_id = os.environ['b2_application_key_id']
b2_application_key = os.environ['b2_application_key']

In [16]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = b2_application_key_id
application_key = b2_application_key
b2_api.authorize_account("production", application_key_id, application_key)
file_info = {'how': 'good-file'}
b2_bucket = b2_api.get_bucket_by_name('malaya-speech-model')

In [17]:
outPutname = 'language-model/bahasa-manglish-combined/model.trie.klm'
b2_bucket.upload_local_file(
    local_file='out-mixed.trie.klm',
    file_name=outPutname,
    file_infos=file_info,
)